In [25]:
from gensim.models.doc2vec import  Doc2Vec, TaggedDocument
from nltk import word_tokenize
import pandas as pd  
import string 

from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

In [37]:
input_data7 = pd.read_csv('input_data7.csv', header = 0,
                          usecols= ['patent_id', 'combined'],
                          dtype = {'patent_id':object},
                          encoding = 'ISO-8859-1')

In [38]:
my_columns = input_data7['patent_id'].tolist() # create patent list
corpus = input_data7['combined'].tolist()
patent_numbers = '|'.join(my_columns)

In [39]:
#preprocessing 


patent_terms = pd.read_csv('patent_terms.csv', encoding = 'ISO-8859-1')
patent_terms = patent_terms.rename(columns={0:'terms'})
patent_terms_append = ['claim','claimed','according','selected','comprising','consisting','means','le','wt','group','thereof','weight']
patent_terms = patent_terms.append(pd.DataFrame(patent_terms_append, columns=['terms']), ignore_index=True)
patent_terms = patent_terms.drop_duplicates()
patent_terms.to_csv('patent_terms.csv', index=False)

jargon = set(patent_terms.ix[:, 0])  # build a "patent jargon" lexical
''' jargon removes word "containing", we want to weigh lignosulfonate which follows!'''

## Punctuation
# nltk.download('punkt')
punctuations = set(string.punctuation)
punctuations.remove('-') # maybe remove hyphens
#punctuations.remove('/')

from nltk.stem.wordnet import WordNetLemmatizer
lemma = WordNetLemmatizer()
word_len = 2
def clean(doc):
    number_free = ''.join([c for c in doc if c not in "1234567890"])
    words = [word.strip(string.punctuation) for word in number_free.split(" ")]
    filtered = [f for f in words if f and f.lower() not in stop_words]
    undo = "".join([" "+i if not i.startswith("'") and i not in string.punctuation else i for i in filtered]).strip()
    punc_free = ''.join(ch for ch in undo if ch not in punctuations)
    smallword_free = ' '.join([w for w in punc_free.split() if len(w) > word_len])
    lemmatized = " ".join(lemma.lemmatize(word) for word in smallword_free.split())
    jargon_free = " " .join([j for j in lemmatized.lower().split() if j not in jargon])
    return jargon_free

corpus_clean = [clean(doc) for doc in corpus] # list of sentance strings
#corpus_tokenize = [clean(doc).split() for doc in corpus]  # list of string words

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  # This is added back by InteractiveShellApp.init_path()


In [40]:
corpus_clean[0]

'plywood adhesive phenolic plywood adhesive containing lignosulfonates trialkyl phosphate alkyltituents carbon atom use curtain coater described process applying phenol-formaldehyde resin adhesive containing amylaceous proteinaceous material plywood veneer using curtain coater improvement comprises adding least percent based upon weight phenol-formaldehyde resin said adhesive sodium base lignosulfonate percent based upon weight phenol-formaldehyde resin trialkyl phosphate alkyltituents carbon atomsa process according claim wherein trialkyl phosphate tributyl phosphatea process according claim wherein sodium base lignosulfonate added amount weight percenta process according claim wherein trialkyl phosphate tributyl phosphate added amount percenta process according claim wherein sodium lignosulfonate added phenol formaldehyde mixture condensation phenol formaldehyde preparation phenol-formaldehyde resina process according claim wherein lignosulfonate added amount weight percent trialkyl 

In [6]:
data = data_df['combined'].tolist()

In [41]:
tagged_data = [TaggedDocument(words= word_tokenize(_d.lower()), tags = [str(i)])
              for i, _d in enumerate(corpus_clean)]

#Need to figure out how to make the patent ID the document tag
# tagged_data = [TaggedDocument(words= word_tokenize(_d.lower()), tags = [str(i)])
#               for i, _d in enumerate(data)]








In [42]:
len(tagged_data)

5651

In [48]:
max_epochs = 100
vec_size = 300
alpha = 0.025

model = Doc2Vec(size= vec_size,
               alpha = alpha,
                min_alpha = 0.0025,
                min_count = 1, 
                dm = 1
               )

model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("patent_test_d2v.model")
print("Model Saved")

C:\ProgramData\Anaconda3\lib\site-packages\gensim\models\doc2vec.py:580: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


iteration 0


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


iteration 1
iteration 2
iteration 3


KeyboardInterrupt: 

In [53]:
modell = Doc2Vec.load('C:/Users/ravi/Documents/Spring2019/IndependentStudyPatentProject/sam_thesis_project/patentd2v_model_files')

PermissionError: [Errno 13] Permission denied: 'C:/Users/ravi/Documents/Spring2019/IndependentStudyPatentProject/sam_thesis_project/patentd2v_model_files'